### Disclaimer
Distribution authorized to U.S. Government agencies and their contractors. Other requests for this document shall be referred to the MIT Lincoln Laboratory Technology Office.

This material is based upon work supported by the Under Secretary of Defense for Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions, findings, conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the Under Secretary of Defense for Research and Engineering.

© 2019 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part 252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed above. Use of this work other than as specifically authorized by the U.S. Government may violate any copyrights that exist in this work.

# Collect semantic segmentation training data


### Contents
- [Configure Environments](#Configure-Environments)
- [Create Data Directory](#Create-Data-Directory)
- [Collect Images](#Collect-Images)

In [ ]:
from gym import spaces
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv
import time
from pathlib import Path
import matplotlib.pyplot as plt

from tesse.msgs import *

%load_ext autoreload
%autoreload 2
from tesse_semantic_segmentation.data import RGBSegmentationEnv
from tesse_gym.core.utils import set_all_camera_params, get_network_config

# Configure Environments

In [ ]:
filename = Path("/home/za27933/tess/goseek-challenge/simulator/goseek-v0.1.0.x86_64")
assert filename.exists(), "Give valid file path!"

total_timesteps = 10000
scene_id = [1, 2, 3, 4, 5]
n_target_types = 5
n_targets = [10] * len(scene_id)
n_environments = len(scene_id)


def make_unity_env(filename, num_env):
    """ Create a wrapped Unity environment. """

    def make_env(rank):
        def _thunk():
            env = RGBSegmentationEnv(
                str(filename),
                network_config=get_network_config(worker_id=rank),
                n_targets=n_targets[rank],
                scene_id=scene_id[rank],
                n_target_types=n_target_types,
                init_hook=set_all_camera_params,
                step_rate=20
            )
            return env

        return _thunk

    return SubprocVecEnv([make_env(i) for i in range(num_env)])

In [ ]:
env = make_unity_env(filename, n_environments)

# Create Data Directory 

Data will be saved in the following stucture 
```
- dataset_root 
    \- rgb 
        \- img_1.png
         - img_2.png
         ...
         - img_n.png
    \- segmentation
        \- img_1.png
         - img_2.png
         ...
         - img_n.png
```

In [ ]:
dataset_root = "goseek-v0.1.0-v2"

rgb_data_dir = Path(f"./data/{dataset_root}/rgb")
rgb_data_dir.mkdir(exist_ok=True, parents=True)

segmentation_data_dir = Path(f"./data/{dataset_root}/segmentation")
segmentation_data_dir.mkdir(exist_ok=True, parents=True)

# Collect Images 

In [ ]:
for i in range(total_timesteps):
    obs = env.reset()

    for j in range(obs.shape[0]):
        rgb = obs[j, ..., :3].astype(np.uint8)
        seg = obs[j, ..., 3:].astype(np.uint8)

        plt.imsave(rgb_data_dir / f"img_{i}_scene_{j}.png", rgb)
        plt.imsave(segmentation_data_dir / f"img_{i}_scene_{j}.png", seg)